In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from scipy import stats

# Configuración
plt.style.use('seaborn-v0_8-whitegrid')
pd.set_option('display.max_columns', 50)

# Rutas
BASE_DIR = Path().resolve().parent
DATA_PATH = BASE_DIR / 'data' / 'gold' / 'model' / 'regression_timeseries_dataset.parquet'

In [ ]:
# Cargar datos
df = pd.read_parquet(DATA_PATH)
print(f"Shape: {df.shape}")
print(f"\nColumnas: {df.columns.tolist()}")
df.head(10)

In [ ]:
# Info general
print("=" * 60)
print("INFO GENERAL")
print("=" * 60)
print(f"\nTotal meses: {len(df)}")

if 'fecha' in df.columns:
    df['fecha'] = pd.to_datetime(df['fecha'])
    print(f"Período: {df['fecha'].min()} a {df['fecha'].max()}")
elif 'anio_mes' in df.columns:
    print(f"Período: {df['anio_mes'].min()} a {df['anio_mes'].max()}")

print(f"\nEstadísticas del target:")
print(df['total_delitos'].describe())

## 1. Serie Temporal Principal

In [ ]:
# Preparar índice temporal
if 'fecha' in df.columns:
    df = df.sort_values('fecha')
    df = df.set_index('fecha')
elif 'anio_mes' in df.columns:
    df['fecha'] = pd.to_datetime(df['anio_mes'])
    df = df.sort_values('fecha')
    df = df.set_index('fecha')

# Visualización de la serie
fig, axes = plt.subplots(2, 1, figsize=(16, 10))

# Serie original
axes[0].plot(df.index, df['total_delitos'], linewidth=1.5, color='steelblue')
axes[0].set_title('Serie Temporal: Total Delitos Mensuales (Departamento)', fontsize=14)
axes[0].set_xlabel('Fecha')
axes[0].set_ylabel('Total Delitos')
axes[0].grid(True, alpha=0.3)

# Añadir media móvil
if len(df) > 12:
    rolling_12 = df['total_delitos'].rolling(window=12).mean()
    axes[0].plot(df.index, rolling_12, linewidth=2, color='red', label='Media móvil 12 meses')
    axes[0].legend()

# Tasa global (si existe)
if 'tasa_global' in df.columns:
    axes[1].plot(df.index, df['tasa_global'], linewidth=1.5, color='coral')
    axes[1].set_title('Serie Temporal: Tasa Global (por 100k habitantes)', fontsize=14)
    axes[1].set_xlabel('Fecha')
    axes[1].set_ylabel('Tasa')
    axes[1].grid(True, alpha=0.3)
else:
    axes[1].set_visible(False)

plt.tight_layout()
plt.show()

## 2. Descomposición de la Serie

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

# Descomposición estacional
try:
    decomposition = seasonal_decompose(df['total_delitos'].dropna(), model='additive', period=12)
    
    fig, axes = plt.subplots(4, 1, figsize=(16, 12))
    
    decomposition.observed.plot(ax=axes[0], title='Observado')
    decomposition.trend.plot(ax=axes[1], title='Tendencia')
    decomposition.seasonal.plot(ax=axes[2], title='Estacionalidad')
    decomposition.resid.plot(ax=axes[3], title='Residuos')
    
    for ax in axes:
        ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Fuerza de estacionalidad
    var_seasonal = decomposition.seasonal.var()
    var_residual = decomposition.resid.dropna().var()
    strength_seasonal = 1 - (var_residual / (var_seasonal + var_residual))
    print(f"\nFuerza de estacionalidad: {strength_seasonal:.3f}")
    print("(> 0.6 indica estacionalidad significativa)")
    
except Exception as e:
    print(f"No se pudo descomponer la serie: {e}")

## 3. Test de Estacionariedad

In [ ]:
from statsmodels.tsa.stattools import adfuller, kpss

print("=" * 60)
print("TESTS DE ESTACIONARIEDAD")
print("=" * 60)

# ADF Test
try:
    adf_result = adfuller(df['total_delitos'].dropna())
    print(f"\nADF Test (H0: Serie tiene raíz unitaria / no estacionaria):")
    print(f"  Estadístico: {adf_result[0]:.4f}")
    print(f"  p-value: {adf_result[1]:.4f}")
    print(f"  Valores críticos: {adf_result[4]}")
    if adf_result[1] < 0.05:
        print(f"  ✅ Serie es ESTACIONARIA (rechazamos H0)")
    else:
        print(f"  ⚠️ Serie NO es estacionaria (no rechazamos H0)")
except Exception as e:
    print(f"Error en ADF: {e}")

# KPSS Test
try:
    kpss_result = kpss(df['total_delitos'].dropna(), regression='c')
    print(f"\nKPSS Test (H0: Serie es estacionaria):")
    print(f"  Estadístico: {kpss_result[0]:.4f}")
    print(f"  p-value: {kpss_result[1]:.4f}")
    if kpss_result[1] > 0.05:
        print(f"  ✅ Serie es ESTACIONARIA (no rechazamos H0)")
    else:
        print(f"  ⚠️ Serie NO es estacionaria (rechazamos H0)")
except Exception as e:
    print(f"Error en KPSS: {e}")

## 4. Autocorrelación

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

fig, axes = plt.subplots(1, 2, figsize=(16, 5))

# ACF
plot_acf(df['total_delitos'].dropna(), lags=36, ax=axes[0])
axes[0].set_title('Función de Autocorrelación (ACF)')

# PACF
plot_pacf(df['total_delitos'].dropna(), lags=36, ax=axes[1])
axes[1].set_title('Función de Autocorrelación Parcial (PACF)')

plt.tight_layout()
plt.show()

print("\nInterpretación:")
print("- Picos significativos en lag 12 indican estacionalidad anual")
print("- Decaimiento lento en ACF indica tendencia")
print("- PACF ayuda a identificar orden AR del modelo")

## 5. Features de Lag Existentes

In [ ]:
# Verificar features de lag
lag_cols = [c for c in df.columns if 'lag' in c.lower()]
roll_cols = [c for c in df.columns if 'roll' in c.lower()]
pct_cols = [c for c in df.columns if 'pct' in c.lower()]

print("=" * 60)
print("FEATURES TEMPORALES EXISTENTES")
print("=" * 60)

print(f"\nLag features: {lag_cols}")
print(f"Rolling features: {roll_cols}")
print(f"Percent change features: {pct_cols}")

# Estadísticas de lag features
if lag_cols:
    print(f"\nEstadísticas de lag features:")
    print(df[lag_cols].describe().round(2))

In [ ]:
# Correlación de lags con target
if lag_cols:
    fig, ax = plt.subplots(figsize=(10, 6))
    
    correlations = df[['total_delitos'] + lag_cols].corr()['total_delitos'].drop('total_delitos')
    correlations.plot(kind='bar', ax=ax, color='steelblue')
    ax.set_title('Correlación de Lag Features con total_delitos')
    ax.set_ylabel('Correlación')
    ax.axhline(0, color='black', linewidth=0.5)
    ax.tick_params(axis='x', rotation=45)
    
    for i, v in enumerate(correlations):
        ax.text(i, v + 0.02, f'{v:.2f}', ha='center', fontsize=9)
    
    plt.tight_layout()
    plt.show()

## 6. Estacionalidad Mensual

In [ ]:
# Patrón estacional por mes
if 'mes' in df.columns:
    df_reset = df.reset_index()
else:
    df_reset = df.reset_index()
    df_reset['mes'] = df_reset['fecha'].dt.month if 'fecha' in df_reset.columns else df_reset.index.month

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Boxplot por mes
df_reset.boxplot(column='total_delitos', by='mes', ax=axes[0])
axes[0].set_title('Distribución por Mes')
axes[0].set_xlabel('Mes')
axes[0].set_ylabel('Total Delitos')
plt.suptitle('')

# Promedio por mes
monthly_mean = df_reset.groupby('mes')['total_delitos'].mean()
axes[1].bar(monthly_mean.index, monthly_mean.values, color='coral')
axes[1].set_xlabel('Mes')
axes[1].set_ylabel('Promedio Delitos')
axes[1].set_title('Promedio de Delitos por Mes')
axes[1].set_xticks(range(1, 13))

plt.tight_layout()
plt.show()

print(f"\nMes con más delitos: {monthly_mean.idxmax()} (promedio: {monthly_mean.max():.0f})")
print(f"Mes con menos delitos: {monthly_mean.idxmin()} (promedio: {monthly_mean.min():.0f})")

## 7. Conclusiones y Recomendaciones

In [ ]:
print("=" * 60)
print("CONCLUSIONES")
print("=" * 60)

print(f"""
SERIE TEMPORAL:
  - Total observaciones: {len(df)}
  - Media: {df['total_delitos'].mean():.0f} delitos/mes
  - Std: {df['total_delitos'].std():.0f}
  - Rango: {df['total_delitos'].min():.0f} - {df['total_delitos'].max():.0f}

FEATURES EXISTENTES:
  - Lag features: {len(lag_cols)}
  - Rolling features: {len(roll_cols)}
  - Pct change features: {len(pct_cols)}

CARACTERÍSTICAS:
  - {'Estacionalidad anual detectada' if lag_cols else 'Verificar estacionalidad'}
  - Dataset pequeño ({len(df)} obs) - cuidado con overfitting
""")

print("\n" + "=" * 60)
print("RECOMENDACIONES PARA MODELO")
print("=" * 60)
print("""
1. MODELOS SUGERIDOS (serie temporal):
   - SARIMA / SARIMAX (estacionalidad + exógenas)
   - Prophet (robusto, maneja festivos)
   - XGBoost con features de lag
   - LSTM (si hay suficientes datos)

2. FEATURES ADICIONALES:
   - Festivos colombianos
   - Eventos especiales (elecciones, COVID)
   - Variables macroeconómicas (desempleo)

3. VALIDACIÓN:
   - Walk-forward validation
   - Train hasta 2023, test en 2024-2025
   - NO usar KFold (rompe temporalidad)

4. MÉTRICAS:
   - RMSE, MAE
   - MAPE (interpretable)
   - Comparar con baseline (naive, media móvil)

5. CONSIDERACIONES:
   - Dataset pequeño (< 200 obs)
   - Evitar modelos muy complejos
   - Preferir modelos interpretables
""")